## Natural Language Processing

The text file "book.txt" is used as the dataset and is used to develop a model to understand the story and hence fill up with words auto generated.

The file is read and the contents are displayed till 200 characters

In [3]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

in_filename = 'book.txt'
doc = load_doc(in_filename)
print(doc[:200])

BOOK I.

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what


The content is cleaned and is now referred to as "clean text". Here all punctionations are removed, all '-' are replaced with spaces and any now alaphabetical word is removed. Along with that the entire dataset is split based on white spaces and hence a linesr list is obtained

In [8]:
import string


def clean_doc(doc):

    doc = doc.replace('--', ' ')
    tokens = doc.split()
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens

tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['book', 'i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight', 'of', 'us', 'from', 'a', 'distance', 'as', 'we', 'were', 'starting', 'on', 'our', 'way', 'home', 'and', 'told', 'his', 'servant', 'to', 'run', 'and', 'bid',

The tokens are iterated over a list of 50 input and 1 output and stored in the 

In [9]:
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 173


In [13]:
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()
    
out_filename = 'book_clean.txt'
save_doc(sequences, out_filename)

### Training the data

This shall involve training the NLP model using Embedding Layer to learn the representation of words, and a Long Short-Term Memory (LSTM) recurrent neural network to learn to predict words based on their context.

It has a few unique characteristics:

* It uses a distributed representation for words so that different words with similar meanings will have a similar representation.
* It learns the representation at the same time as learning the model.
* It learns to predict the probability for the next word using the context of the last 100 words.

In [14]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load
in_filename = 'book_clean.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

Each word is associated an integer and the ouput is predicted using this sequence of integers. The Tokenizer class in Keras API is used to do this encoding. The total size of the vocabulary can be a maximum of the total number of words and need to be found out.

In [17]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
vocab_size = len(tokenizer.word_index) + 1

The keras package is used again to hot encode the words, with 0 for the number of values and  to indicate the specific word at the index of the words integer value. x and y are differentiated as input and output respectively.

In [20]:
import numpy as np
from keras.utils import to_categorical

sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

### Fit Model

The model can now be fit on the training data. The learned embedding needs to know the size of the vocabulary and the length of input sequences as previously discussed. It also has a parameter to specify how many dimensions will be used to represent each word. That is, the size of the embedding vector space.

The Long Short Term Memory(LSTM) from keras package is used. 2 LSTM layers with 100 neurons are connected in each and predict the next word.

In [27]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Dense

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 50)            6300      
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 126)               12726     
Total params: 169,926
Trainable params: 169,926
Non-trainable params: 0
_________________________________________________________________
None


Finally the model is compiled and saved for use

In [28]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
2/2 [==============================] - 8s 162ms/step - loss: 4.8361 - accuracy: 0.0142
Epoch 2/100
2/2 [==============================] - 0s 116ms/step - loss: 4.8313 - accuracy: 0.0761
Epoch 3/100
2/2 [==============================] - 0s 117ms/step - loss: 4.8237 - accuracy: 0.0735
Epoch 4/100
2/2 [==============================] - 0s 112ms/step - loss: 4.8090 - accuracy: 0.0735
Epoch 5/100
2/2 [==============================] - 0s 105ms/step - loss: 4.7723 - accuracy: 0.0735
Epoch 6/100
2/2 [==============================] - 0s 114ms/step - loss: 4.6691 - accuracy: 0.0735
Epoch 7/100
2/2 [==============================] - 0s 118ms/step - loss: 4.5411 - accuracy: 0.0735
Epoch 8/100
2/2 [==============================] - 0s 109ms/step - loss: 4.5398 - accuracy: 0.0761
Epoch 9/100
2/2 [==============================] - 0s 134ms/step - loss: 4.5059 - accuracy: 0.0735
Epoch 10/100
2/2 [==============================] - 0s 118ms/step - loss: 4.4185 - accuracy: 0.0787
Epoch 11/

2/2 [==============================] - 0s 114ms/step - loss: 4.0837 - accuracy: 0.0851
Epoch 84/100
2/2 [==============================] - 0s 102ms/step - loss: 4.0794 - accuracy: 0.0903
Epoch 85/100
2/2 [==============================] - 0s 117ms/step - loss: 4.0553 - accuracy: 0.0851
Epoch 86/100
2/2 [==============================] - 0s 110ms/step - loss: 4.0167 - accuracy: 0.0877
Epoch 87/100
2/2 [==============================] - 0s 125ms/step - loss: 4.0182 - accuracy: 0.0851
Epoch 88/100
2/2 [==============================] - 0s 121ms/step - loss: 3.9608 - accuracy: 0.0981
Epoch 89/100
2/2 [==============================] - 0s 125ms/step - loss: 3.9602 - accuracy: 0.0903
Epoch 90/100
2/2 [==============================] - 0s 120ms/step - loss: 3.9176 - accuracy: 0.0864
Epoch 91/100
2/2 [==============================] - 0s 123ms/step - loss: 3.9038 - accuracy: 0.0903
Epoch 92/100
2/2 [==============================] - 0s 109ms/step - loss: 3.8636 - accuracy: 0.0955
Epoch 93/100


In [31]:
import pickle
model.save('model.h5')
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))